In [0]:
# PROBLEM STATEMENT:

In [0]:
'''
Explore the datasets to perform the following task

Tasks:


1. calculate average meter usage by ACORN(A,B,C,D) group on the month(Jan, Feb, Mar) using 30 mins data. use data available for 2013 and 2014. Plot average monthly usage graph for ACORN A,B,C,D if possible

2. use meter usage data for 2013 and 2014 for ACORN groups A,B,C,D and identify which day of week has the highest average electricity usage

3. for year 2013 please identify which month average usage is maximum across ACORN groups (A,B,C,D)

4. use a weather dataset which is available and evaluate how weather impacts meter electricity usage

6. use the data and try to find one useful insight which will help business to make better decisions

'''

In [0]:
# CREATING A PYSPARK SESSION

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('meters').getOrCreate()
spark

Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# IMPORTING DATA INTO DATAFRAMES FROM DBFS

In [0]:
df_households = spark.read.csv("dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/informations_households-1.csv", header=True,inferSchema = True)
df_weather_hourly =   spark.read.csv("dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/weather_hourly_darksky-1.csv", header=True,inferSchema = True)
df_acorn = spark.read.csv("dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/acorn_details-2.csv", header=True,inferSchema = True)
df_weather_daily =  spark.read.csv("dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/weather_daily_darksky-1.csv",  header=True,inferSchema = True)

In [0]:
# CHECK THE SCHEMA FOR DATATYPES AND HAVE A LOOK AT THE HOUSHOLD DATAFRAME

In [0]:
df_households.printSchema()
df_households.limit(2).toPandas()

root
-- LCLid: string (nullable = true)
-- stdorToU: string (nullable = true)
-- Acorn: string (nullable = true)
-- Acorn_grouped: string (nullable = true)
-- file: string (nullable = true)

Out[7]:

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0


In [0]:
# CHECK THE HOUSEHOLD DATA FRAME TO EXTRACT THE BLOCK FILES WHICH HOLDS THE DATA FOR ACORN-A,ACORN-B,ACORN-C AND ACORN-D.

In [0]:
# ACORN-A GROUP HAS ITS ENERGY CONSUMPTION FILES IN BLOCK_0,BLOCK_1,BLOCK_2 AND BLOCK_3.

In [0]:
df_households.filter(df_households['Acorn'] == 'ACORN-A').select('file').distinct().show()

+-------+
 file|
+-------+
block_1|
block_3|
block_0|
block_2|
+-------+

In [0]:
# ACORN-B GROUP HAS ITS ENERGY CONSUMPTION FILES IN  BLOCK_3.

In [0]:
df_households.filter(df_households['Acorn'] == 'ACORN-B').select('file').distinct().show()

+-------+
 file|
+-------+
block_3|
+-------+

In [0]:
# ACORN-C GROUP HAS ITS ENERGY CONSUMPTION FILES IN BLOCK_3,BLOCK_4,BLOCK_5 AND BLOCK_6.

In [0]:
df_households.filter(df_households['Acorn'] == 'ACORN-C').select('file').distinct().show()

+-------+
 file|
+-------+
block_4|
block_3|
block_5|
block_6|
+-------+

In [0]:
# ACORN-D GROUP HAS ITS ENERGY CONSUMPTION FILES IN BLOCK_6,BLOCK_7,BLOCK_8 ,BLOCK_9,,BLOCK_10,,BLOCK_11 AND BLOCK_12

In [0]:
df_households.filter(df_households['Acorn'] == 'ACORN-D').select('file').distinct().show()

+--------+
 file|
+--------+
block_12|
 block_8|
block_10|
 block_9|
block_11|
 block_6|
 block_7|
+--------+

In [0]:
# CHECKING THE WEATHER HOURLY TABLE FOR DATA TYPES AND FIRST TWO ROWS.

In [0]:
df_weather_hourly.printSchema()
df_weather_hourly.limit(2).toPandas()

root
-- visibility: double (nullable = true)
-- windBearing: integer (nullable = true)
-- temperature: double (nullable = true)
-- time: string (nullable = true)
-- dewPoint: double (nullable = true)
-- pressure: double (nullable = true)
-- apparentTemperature: double (nullable = true)
-- windSpeed: double (nullable = true)
-- precipType: string (nullable = true)
-- icon: string (nullable = true)
-- humidity: double (nullable = true)
-- summary: string (nullable = true)

Out[12]:

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy


In [0]:
# HERE TIME COLUMN IS STRING. CASTING IT INTO TIMESTAMP AND RENAMING THE COLUMN.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
df_weather_hourly_new = df_weather_hourly.withColumn('time',to_timestamp(df_weather_hourly['time'],'yyyy-MM-dd HH:mm:ss')).withColumnRenamed('time','Timestamp')

In [0]:
df_weather_hourly_new.printSchema()

root
-- visibility: double (nullable = true)
-- windBearing: integer (nullable = true)
-- temperature: double (nullable = true)
-- Timestamp: timestamp (nullable = true)
-- dewPoint: double (nullable = true)
-- pressure: double (nullable = true)
-- apparentTemperature: double (nullable = true)
-- windSpeed: double (nullable = true)
-- precipType: string (nullable = true)
-- icon: string (nullable = true)
-- humidity: double (nullable = true)
-- summary: string (nullable = true)

In [0]:

df_weather_hourly_new.limit(2).toPandas()

Out[16]:

,visibility,windBearing,temperature,Timestamp,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy


In [0]:
# FROM THE TIMESTAMP COLUMN EXTRACTING YEAR, MONTH AND DATE.

In [0]:
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col

df_weather_hourly_new = df_weather_hourly_new.withColumn('Year1',year(df_weather_hourly_new['Timestamp'])).withColumn('Month',month(df_weather_hourly_new['Timestamp'])).withColumn('Date',dayofmonth
(df_weather_hourly_new['Timestamp']))
df_weather_hourly_new.limit(2).toPandas()

Out[17]:

,visibility,windBearing,temperature,Timestamp,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary,Year1,Month,Date
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy,2011,11,11
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy,2011,11,11


In [0]:
# EXTRACTING THE NAME OF THE DAY OF THE WEEK AND THE MONTH NAME FROM TIMESTAMP

In [0]:
df_weather_hourly_new = df_weather_hourly_new.withColumn('Week_day',date_format(col('Timestamp'),'EEEE')). \
withColumn('Month_name',date_format(col('Timestamp'),'MMMM'))
df_weather_hourly_new.limit(2).toPandas()

Out[18]:

,visibility,windBearing,temperature,Timestamp,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary,Year1,Month,Date,Week_day,Month_name
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy,2011,11,11,Friday,November
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy,2011,11,11,Friday,November


In [0]:
# CHECKING THE COLUMNS FOR WEATHER DATA DAILY DATASET AND CHECKING THE SCHEMA.

In [0]:
df_weather_daily.limit(2).toPandas()

Out[19]:

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,rain,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,2011-11-11 00:00:00,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,rain,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,2011-12-11 00:00:00,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53


In [0]:
df_weather_daily.printSchema()

root
-- temperatureMax: double (nullable = true)
-- temperatureMaxTime: string (nullable = true)
-- windBearing: integer (nullable = true)
-- icon: string (nullable = true)
-- dewPoint: double (nullable = true)
-- temperatureMinTime: string (nullable = true)
-- cloudCover: double (nullable = true)
-- windSpeed: double (nullable = true)
-- pressure: double (nullable = true)
-- apparentTemperatureMinTime: string (nullable = true)
-- apparentTemperatureHigh: double (nullable = true)
-- precipType: string (nullable = true)
-- visibility: double (nullable = true)
-- humidity: double (nullable = true)
-- apparentTemperatureHighTime: string (nullable = true)
-- apparentTemperatureLow: double (nullable = true)
-- apparentTemperatureMax: double (nullable = true)
-- uvIndex: double (nullable = true)
-- time: string (nullable = true)
-- sunsetTime: string (nullable = true)
-- temperatureLow: double (nullable = true)
-- temperatureMin: double (nullable = true)
-- temperatureHigh: double (nullable = true)
-- sunriseTime: string (nullable = true)
-- temperatureHighTime: string (nullable = true)
-- uvIndexTime: string (nullable = true)
-- summary: string (nullable = true)
-- temperatureLowTime: string (nullable = true)
-- apparentTemperatureMin: double (nullable = true)
-- apparentTemperatureMaxTime: string (nullable = true)
-- apparentTemperatureLowTime: string (nullable = true)
-- moonPhase: double (nullable = true)

In [0]:
# CHECKING THE ACORN DATASET FOR THE COLUMNS AND CHECKING THE SCHEMA.

In [0]:
df_acorn.limit(2).toPandas()

Out[20]:

,MAIN CATEGORIES,CATEGORIES,REFERENCE,ACORN-A,ACORN-B,ACORN-C,ACORN-D,ACORN-E,ACORN-F,ACORN-G,ACORN-H,ACORN-I,ACORN-J,ACORN-K,ACORN-L,ACORN-M,ACORN-N,ACORN-O,ACORN-P,ACORN-Q
0,POPULATION,Age,Age 0-4,77.0,83.0,72.0,100.0,120.0,77.0,97.0,97.0,63.0,119.0,67.0,114.0,113.0,89.0,123.0,138.0,133.0
1,POPULATION,Age,Age 5-17,117.0,109.0,87.0,69.0,94.0,95.0,102.0,106.0,67.0,95.0,64.0,108.0,116.0,86.0,89.0,136.0,106.0


In [0]:
df_acorn.printSchema()

root
-- MAIN CATEGORIES: string (nullable = true)
-- CATEGORIES: string (nullable = true)
-- REFERENCE: string (nullable = true)
-- ACORN-A: double (nullable = true)
-- ACORN-B: double (nullable = true)
-- ACORN-C: double (nullable = true)
-- ACORN-D: double (nullable = true)
-- ACORN-E: double (nullable = true)
-- ACORN-F: double (nullable = true)
-- ACORN-G: double (nullable = true)
-- ACORN-H: double (nullable = true)
-- ACORN-I: double (nullable = true)
-- ACORN-J: double (nullable = true)
-- ACORN-K: double (nullable = true)
-- ACORN-L: double (nullable = true)
-- ACORN-M: double (nullable = true)
-- ACORN-N: double (nullable = true)
-- ACORN-O: double (nullable = true)
-- ACORN-P: double (nullable = true)
-- ACORN-Q: double (nullable = true)

In [0]:
'''
READING ALL THE DATASETS OF BLOCKS FROM 1 TO 12( THOSE NEEDED FOR ACORN A,B,C AND D) 
INTO A SINGLE DATASET AND CHECKING THE COLUMNS AND SCHEMA.
'''

In [0]:
paths_new = ["dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_0.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_1.csv", "dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_2.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_3.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_4.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_5.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_6.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_7.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_8.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_9.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_10.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_11.csv","dbfs:/FileStore/shared_uploads/himadri_88@yahoo.com/block_12.csv"]

df_all_blocks = spark.read.csv(paths_new,header = True, inferSchema = True)

In [0]:
df_all_blocks.count()

Out[22]: 19192296

In [0]:
df_all_blocks.limit(3).toPandas()

Out[23]:

,LCLid,tstp,energy(kWh/hh)
0,MAC000291,2012-03-02 09:30:00.0000000,1.158
1,MAC000291,2012-03-02 10:00:00.0000000,1.499
2,MAC000291,2012-03-02 10:30:00.0000000,1.433


In [0]:
'''
CHECKING THE SCHEMA. 
TSTP IS IN STRING. TO BE CONVERTED INTO TIME STAMP.
energy(kWh/hh) IN STRING. TO BE CASTED TO INTEGER AND THEN RENAMED TO KWh
'''

In [0]:
df_all_blocks.printSchema()

root
-- LCLid: string (nullable = true)
-- tstp: string (nullable = true)
-- energy(kWh/hh): string (nullable = true)

In [0]:
df_all_blocks= df_all_blocks.withColumn('tstp',to_timestamp(df_all_blocks['tstp'],'yyyy-MM-dd HH:mm:ss.SSSSSSS')).withColumnRenamed('tstp','Timestamp').withColumnRenamed('energy(kWh/hh)','KWh')
df_all_blocks = df_all_blocks.withColumn('Year',year(df_all_blocks['Timestamp'])).withColumn('Month',month(df_all_blocks['Timestamp'])).withColumn('Day_of_week',dayofweek(df_all_blocks['Timestamp'])) \
.withColumn('Date',dayofmonth(df_all_blocks['Timestamp'])).withColumn('Week_day',date_format(col('Timestamp'),'EEEE')). \
withColumn('Month_name',date_format(col('Timestamp'),'MMMM'))
  
  

In [0]:
df_all_blocks.printSchema()
df_all_blocks.limit(2).toPandas()

root
-- LCLid: string (nullable = true)
-- Timestamp: timestamp (nullable = true)
-- KWh: integer (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Date: integer (nullable = true)
-- Week_day: string (nullable = true)
-- Month_name: string (nullable = true)
-- Day_of_week: integer (nullable = true)

Out[69]:

,LCLid,Timestamp,KWh,Year,Month,Date,Week_day,Month_name,Day_of_week
0,MAC000291,2012-03-02 09:30:00,1,2012,3,2,Friday,March,6
1,MAC000291,2012-03-02 10:00:00,1,2012,3,2,Friday,March,6


In [0]:
from pyspark.sql.types import *
df_all_blocks = df_all_blocks.withColumn('KWh',df_all_blocks['KWh'].cast(IntegerType()))
df_all_blocks.printSchema()

root
-- LCLid: string (nullable = true)
-- Timestamp: timestamp (nullable = true)
-- KWh: integer (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Date: integer (nullable = true)
-- Week_day: string (nullable = true)
-- Month_name: string (nullable = true)
-- Day_of_week: integer (nullable = true)

In [0]:
'''
CHECKING ALL THE NULL VALUES PRESENT IN ANY COLUMN.
'''

In [0]:
from pyspark.sql.functions import *

df_all_blocks.select([count(when(col(c).isNull(),c)).alias (c) for c in df_all_blocks.columns]).toPandas().T


Out[71]:

,0
LCLid,0
Timestamp,0
KWh,649
Year,0
Month,0
Date,0
Week_day,0
Month_name,0
Day_of_week,0


In [0]:
'''
FILLING THE NULL VALUES WITH THE MEAN OF THAT COLUMN.
'''

In [0]:
def fill_with_mean(df,include = set()):
  stats = df.agg(*(avg(c).alias(c) for c in df.columns if c in include))
  return df.na.fill(stats.first().asDict())

df_all_blocks_cleaned = fill_with_mean(df_all_blocks,['KWh'])

In [0]:
df_all_blocks_cleaned.select([count(when(col(c).isNull(),c)).alias (c) for c in df_all_blocks.columns]).toPandas().T

Out[73]:

,0
LCLid,0
Timestamp,0
KWh,0
Year,0
Month,0
Date,0
Week_day,0
Month_name,0
Day_of_week,0


In [0]:
df_all_blocks_cleaned.limit(5).toPandas()

Out[74]:

,LCLid,Timestamp,KWh,Year,Month,Date,Week_day,Month_name,Day_of_week
0,MAC000291,2012-03-02 09:30:00,1,2012,3,2,Friday,March,6
1,MAC000291,2012-03-02 10:00:00,1,2012,3,2,Friday,March,6
2,MAC000291,2012-03-02 10:30:00,1,2012,3,2,Friday,March,6
3,MAC000291,2012-03-02 11:00:00,0,2012,3,2,Friday,March,6
4,MAC000291,2012-03-02 11:30:00,0,2012,3,2,Friday,March,6


In [0]:
'''
JOINING ALL THE BLOCK DATA FRAMES STORED TOGETHER WITH HOUSEHOLDS DATAFRAME ON LCLid.
'''

In [0]:
df_total = df_all_blocks_cleaned.join(df_households,['LCLid'],how = 'left')
df_total.limit(5).toPandas()

Out[75]:

,LCLid,Timestamp,KWh,Year,Month,Date,Week_day,Month_name,Day_of_week,stdorToU,Acorn,Acorn_grouped,file
0,MAC000291,2012-03-02 09:30:00,1,2012,3,2,Friday,March,6,Std,ACORN-D,Affluent,block_12
1,MAC000291,2012-03-02 10:00:00,1,2012,3,2,Friday,March,6,Std,ACORN-D,Affluent,block_12
2,MAC000291,2012-03-02 10:30:00,1,2012,3,2,Friday,March,6,Std,ACORN-D,Affluent,block_12
3,MAC000291,2012-03-02 11:00:00,0,2012,3,2,Friday,March,6,Std,ACORN-D,Affluent,block_12
4,MAC000291,2012-03-02 11:30:00,0,2012,3,2,Friday,March,6,Std,ACORN-D,Affluent,block_12


In [0]:
'''
FOR THE YEAR 2013 AND 2014 CHECKING THE AVG KWh CONSUMPTION OVER THE WEEKDAYS.
'''

In [0]:
# FOR ACORN -A

In [0]:
df_Acorn_A = df_total[df_total.Acorn.isin('ACORN-A')][df_total.Year.isin(2013,2014)]
df_Acorn_A.groupBy('Week_day','Day_of_week').mean('KWh').orderBy(col('Day_of_week')).select('Week_day','avg(KWh)').toPandas()

Out[84]:

,Week_day,avg(KWh)
0,Sunday,0.125369
1,Monday,0.114230
2,Tuesday,0.111810
3,Wednesday,0.111590
4,Thursday,0.112297
5,Friday,0.110237
6,Saturday,0.117577


In [0]:
print('Maximum eneergy consumption for ACORN-A is for the weekday : ')
df_Acorn_A.groupBy('Week_day').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Maximum eneergy consumption for ACORN-A is for the weekday : 
Out[85]:

,Week_day,avg(KWh)
0,Sunday,0.125369


In [0]:
# FOR ACORN-B

In [0]:
df_Acorn_B = df_total[df_total.Acorn.isin('ACORN-B')][df_total.Year.isin(2013,2014)]
df_Acorn_B.groupBy('Week_day','Day_of_week').mean('KWh').orderBy(col('Day_of_week')).select('Week_day','avg(KWh)').toPandas()

Out[86]:

,Week_day,avg(KWh)
0,Sunday,0.049251
1,Monday,0.035860
2,Tuesday,0.032482
3,Wednesday,0.035677
4,Thursday,0.033562
5,Friday,0.038797
6,Saturday,0.041379


In [0]:
print('Maximum eneergy consumption for ACORN-B is for the weekday : ')
df_Acorn_B.groupBy('Week_day').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Maximum eneergy consumption for ACORN-B is for the weekday : 
Out[87]:

,Week_day,avg(KWh)
0,Sunday,0.049251


In [0]:
# FOR ACORN-C

In [0]:
df_Acorn_C = df_total[df_total.Acorn.isin('ACORN-C')][df_total.Year.isin(2013,2014)]
df_Acorn_C.groupBy('Week_day','Day_of_week').mean('KWh').orderBy(col('Day_of_week')).select('Week_day','avg(KWh)').toPandas()

Out[88]:

,Week_day,avg(KWh)
0,Sunday,0.043419
1,Monday,0.036794
2,Tuesday,0.034637
3,Wednesday,0.033670
4,Thursday,0.033139
5,Friday,0.036544
6,Saturday,0.037490


In [0]:
print('Maximum eneergy consumption for ACORN-C is for the weekday : ')
df_Acorn_C.groupBy('Week_day').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Maximum eneergy consumption for ACORN-C is for the weekday : 
Out[89]:

,Week_day,avg(KWh)
0,Sunday,0.043419


In [0]:
# FOR ACORN-D

In [0]:
df_Acorn_D = df_total[df_total.Acorn.isin('ACORN-D')][df_total.Year.isin(2013,2014)]
df_Acorn_D.groupBy('Week_day','Day_of_week').mean('KWh').orderBy(col('Day_of_week')).select('Week_day','avg(KWh)').toPandas()

Out[90]:

,Week_day,avg(KWh)
0,Sunday,0.064631
1,Monday,0.063214
2,Tuesday,0.062541
3,Wednesday,0.061106
4,Thursday,0.059844
5,Friday,0.059071
6,Saturday,0.059603


In [0]:
print('Maximum eneergy consumption for ACORN-D is for the weekday : ')
df_Acorn_D.groupBy('Week_day').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Maximum eneergy consumption for ACORN-D is for the weekday : 
Out[91]:

,Week_day,avg(KWh)
0,Sunday,0.064631


In [0]:
'''
CHECKING THE MONTH WISE AVERAGE ENERGY CONSUMPTION FOR ACORN A,B,C AND D GROUPS FOR THE YEAR 2013.
'''

In [0]:
# FOR ACORN-A

In [0]:
df_Acorn_A1 = df_total[df_total.Acorn.isin('ACORN-A')][df_total.Year.isin(2013)]
df_Acorn_A1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).select('Month_name','avg(KWh)').toPandas()

Out[92]:

,Month_name,avg(KWh)
0,January,0.173968
1,February,0.160963
2,March,0.159970
3,April,0.108278
4,May,0.082610
5,June,0.067976
6,July,0.053098
7,August,0.061937
8,September,0.082989
9,October,0.091450


In [0]:
print('Highest monthly consumption for ACORN-A in 2013 in the month of :')
df_Acorn_A1.groupBy('Month_name').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Highest monthly consumption for ACORN-A in 2013 in the month of :
Out[51]:

,Month_name,avg(KWh)
0,January,0.173968


In [0]:
a = df_Acorn_A1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month'))
a.registerTempTable("Monthly_energy_usage_2013")
display(sqlContext.sql("select * from Monthly_energy_usage_2013"))

Month_name,Month,avg(KWh)
January,1,0.1739675625016271
February,2,0.16096297615666413
March,3,0.15996994557002944
April,4,0.1082779869907566
May,5,0.08261018167936898
June,6,0.06797574038341983
July,7,0.0530978195317421
August,8,0.061936768054097535
September,9,0.08298884568685719
October,10,0.0914501017954694


In [0]:
# FOR ACORN-B

In [0]:
df_Acorn_B1 = df_total[df_total.Acorn.isin('ACORN-B')][df_total.Year.isin(2013)]
df_Acorn_B1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).select('Month_name','avg(KWh)').toPandas()

Out[94]:

,Month_name,avg(KWh)
0,January,0.051454
1,February,0.044722
2,March,0.053190
3,April,0.029285
4,May,0.023579
5,June,0.019831
6,July,0.011459
7,August,0.012658
8,September,0.028560
9,October,0.036570


In [0]:
print('Highest monthly consumption for ACORN-B in 2013 in the month of :')
df_Acorn_B1.groupBy('Month_name').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Highest monthly consumption for ACORN-B in 2013 in the month of :
Out[60]:

,Month_name,avg(KWh)
0,December,0.065101


In [0]:
df_Acorn_B1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).registerTempTable("Monthly_energy_usage_2013")
display(sqlContext.sql("select * from Monthly_energy_usage_2013"))

Month_name,Month,avg(KWh)
January,1,0.05145437926770256
February,2,0.04472196183967827
March,3,0.05319029746232704
April,4,0.029284949503718495
May,5,0.023579489764485144
June,6,0.019830930461467198
July,7,0.011459150509918189
August,8,0.012657873365258058
September,9,0.02855985416244683
October,10,0.036570340501792115


In [0]:
# FOR ACORN-C

In [0]:
df_Acorn_C1 = df_total[df_total.Acorn.isin('ACORN-C')][df_total.Year.isin(2013)]
df_Acorn_C1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).select('Month_name','avg(KWh)').toPandas()

Out[95]:

,Month_name,avg(KWh)
0,January,0.060429
1,February,0.052517
2,March,0.049210
3,April,0.029645
4,May,0.022392
5,June,0.018642
6,July,0.015935
7,August,0.016095
8,September,0.026474
9,October,0.029824


In [0]:
print('Highest monthly consumption for ACORN-C in 2013 in the month of :')
df_Acorn_C1.groupBy('Month_name').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Highest monthly consumption for ACORN-C in 2013 in the month of :
Out[61]:

,Month_name,avg(KWh)
0,January,0.060429


In [0]:
df_Acorn_C1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).registerTempTable("Monthly_energy_usage_2013")
display(sqlContext.sql("select * from Monthly_energy_usage_2013"))

Month_name,Month,avg(KWh)
January,1,0.060428902662372226
February,2,0.052517209339501054
March,3,0.049210369555233203
April,4,0.029645239940274813
May,5,0.0223919156816571
June,6,0.018641993393787894
July,7,0.015935234523292246
August,8,0.016095248594421782
September,9,0.02647393133084261
October,10,0.02982353936068254


In [0]:
# FOR ACORN-D

In [0]:
df_Acorn_D1 = df_total[df_total.Acorn.isin('ACORN-D')][df_total.Year.isin(2013)]
df_Acorn_D1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).select('Month_name','avg(KWh)').toPandas()

Out[96]:

,Month_name,avg(KWh)
0,January,0.102764
1,February,0.099954
2,March,0.096479
3,April,0.053962
4,May,0.039184
5,June,0.031366
6,July,0.022797
7,August,0.020482
8,September,0.037622
9,October,0.049988


In [0]:
print('Highest monthly consumption for ACORN-D in 2013 in the month of :')
df_Acorn_D1.groupBy('Month_name').mean('KWh').sort(col('avg(KWh)').desc()).limit(1).toPandas()

Highest monthly consumption for ACORN-D in 2013 in the month of :
Out[62]:

,Month_name,avg(KWh)
0,January,0.102764


In [0]:
df_Acorn_D1.groupBy('Month_name','Month').mean('KWh').orderBy(col('Month')).registerTempTable("Monthly_energy_usage_2013")
display(sqlContext.sql("select * from Monthly_energy_usage_2013"))

Month_name,Month,avg(KWh)
January,1,0.10276424839242543
February,2,0.09995443447387621
March,3,0.09647878209767953
April,4,0.05396178610793854
May,5,0.03918392057149776
June,6,0.03136645573585615
July,7,0.02279743819323147
August,8,0.020482359819397566
September,9,0.037621610432622646
October,10,0.04998829850017298


In [0]:
'''
CHECKING THE MONTHLY AVERAGE FOR ACRON A,B,C AND D GROUPS FOR THE YEAR 2013 ONLY FOR THE MONTH JANUARY , FEBRUARY AND MARCH.
'''

In [0]:
# FOR ACRON-A

In [0]:
df_Acorn_A2 = df_total[df_total.Acorn.isin('ACORN-A')][df_total.Year.isin(2013,2014)][df_total.Month_name.isin('January','February','March')]
df_Acorn_A2.groupBy('Month_name').mean('KWh').toPandas()

Out[170]:

,Month_name,avg(KWh)
0,February,0.146463
1,January,0.160091
2,March,0.159970


In [0]:
# FOR ACRON-B

In [0]:
df_Acorn_B2 = df_total[df_total.Acorn.isin('ACORN-B')][df_total.Year.isin(2013,2014)][df_total.Month_name.isin('January','February','March')]
df_Acorn_B2.groupBy('Month_name').mean('KWh').toPandas()

Out[171]:

,Month_name,avg(KWh)
0,February,0.047042
1,January,0.056057
2,March,0.053190


In [0]:
# FOR ACRON-C

In [0]:
df_Acorn_C2 = df_total[df_total.Acorn.isin('ACORN-C')][df_total.Year.isin(2013,2014)][df_total.Month_name.isin('January','February','March')]
df_Acorn_C2.groupBy('Month_name').mean('KWh').toPandas()

Out[172]:

,Month_name,avg(KWh)
0,February,0.049397
1,January,0.054909
2,March,0.049210


In [0]:
# FOR ACRON-D

In [0]:
df_Acorn_D2 = df_total[df_total.Acorn.isin('ACORN-D')][df_total.Year.isin(2013,2014)][df_total.Month_name.isin('January','February','March')]
df_Acorn_D2.groupBy('Month_name').mean('KWh').toPandas()

Out[173]:

,Month_name,avg(KWh)
0,February,0.085707
1,January,0.092267
2,March,0.096479


In [0]:
'''
JOINING THE TOTAL ENERGY CONSUMPTION DATASET WITH THE WEATHER DATASET OVER THE TIMESTAMP COLUMN.
'''

In [0]:
df_conso = df_total.join(df_weather_hourly_new,['Timestamp'],'left').select(['LCLid','KWh','Year',df_total.Month_name,df_total.Week_day,'Acorn','Acorn_grouped','temperature','summary'])
df_conso.limit(2).toPandas()


Out[59]:

,LCLid,KWh,Year,Month_name,Week_day,Acorn,Acorn_grouped,temperature,summary
0,MAC000291,1,2012,March,Friday,ACORN-D,Affluent,NaN,None
1,MAC000291,1,2012,March,Friday,ACORN-D,Affluent,7.93,Foggy


In [0]:
'''
CHECKING THE TOTAL NULL VALUES.
'''

In [0]:
from pyspark.sql.functions import *

df_conso.select([count(when(col(c).isNull(),c)).alias (c) for c in df_conso.columns]).toPandas().T

Out[207]:

,0
LCLid,0
KWh,0
Year,0
Month_name,0
Week_day,0
Acorn,0
Acorn_grouped,0
temperature,9597533


In [0]:
df_conso.select('temperature').count()

Out[208]: 19192296

In [0]:
'''
ALMOST 50% OF THE VALUES IN TEMPERATURE COLUMN IS MISSING.
'''

In [0]:
og_len = df_conso.select('temperature').count()
drop_len = df_conso.select('temperature').na.drop().count()
print("Total Rows Dropped:",og_len-drop_len)
print("Percentage of Rows Dropped", (og_len-drop_len)/og_len)

Total Rows Dropped: 9597533
Percentage of Rows Dropped 0.5000721643726211

In [0]:
'''
CHECKING THE DISTINCT TYPES IN THE SUMMARY COLUMN
'''

In [0]:
summary = df_conso.select('summary').distinct()
summary.toPandas()

Out[220]:

,summary
0,Breezy
1,Windy and Overcast
2,Foggy
3,None
4,Breezy and Partly Cloudy
5,Partly Cloudy
6,Clear
7,Mostly Cloudy
8,Breezy and Overcast
9,Breezy and Mostly Cloudy


In [0]:
df_conso.groupBy('summary').mean('temperature','KWh').na.drop().toPandas()

Out[110]:

,summary,avg(temperature),avg(KWh)
0,Breezy,10.832472,0.062529
1,Windy and Overcast,9.881650,0.132431
2,Foggy,7.458852,0.063039
3,Breezy and Partly Cloudy,11.117180,0.075468
4,Partly Cloudy,12.110342,0.070989
5,Clear,10.440603,0.045242
6,Mostly Cloudy,10.323594,0.080352
7,Breezy and Overcast,11.074756,0.089203
8,Breezy and Mostly Cloudy,11.166066,0.073077
9,Windy,10.480064,0.071033


In [0]:
'''
DROPPING THE NULL VALUES FROM THE TEMPERATURE COLUMN
'''

In [0]:
climate_vs_power = df_conso.groupBy('summary').mean('KWh','temperature').sort(col('avg(KWh)').desc()).na.drop().toPandas()

In [0]:
'''
PLOTTING THE ENERGY CONSUMPTION VS THE CLIMATE CONDITION.
'''

In [0]:
df_conso.groupBy('summary').mean('KWh').registerTempTable("Climate_vs_power_consumption_chart")
display(sqlContext.sql("select * from Climate_vs_power_consumption_chart"))

summary,avg(KWh)
Breezy,0.06252927185402184
Windy and Overcast,0.1324314024390244
Foggy,0.06303930968360498
null,0.06656460571690663
Breezy and Partly Cloudy,0.0754682447684705
Partly Cloudy,0.07098884000798618
Clear,0.045242388218882254
Mostly Cloudy,0.0803520864149503
Breezy and Overcast,0.08920285482427849
Breezy and Mostly Cloudy,0.07307696878230799


In [0]:
'''
PLOTTING THE TEMPERATURE VS THE CLIMATE CONDITION
'''

In [0]:
df_conso.groupBy('summary').mean('temperature').na.drop().registerTempTable("Climate_vs_temperature_chart")
display(sqlContext.sql("select * from Climate_vs_temperature_chart"))

summary,avg(temperature)
Breezy,10.832472151561568
Windy and Overcast,9.881650152439033
Foggy,7.458852105370683
Breezy and Partly Cloudy,11.117179811220394
Partly Cloudy,12.110342307454289
Clear,10.44060325639455
Mostly Cloudy,10.32359365807814
Breezy and Overcast,11.074755820970068
Breezy and Mostly Cloudy,11.166065762735823
Windy,10.48006396792053


In [0]:
'''
PLOTTING AVERAGE POWER CONSUMPTION AND TEMPERATURE WITH RESPECT TO THE CLIMATIC CONDITION.
'''

In [0]:

df_conso.groupBy('summary').mean('KWh','temperature').na.drop().registerTempTable("Climate_vs_power_consumption_chart")
display(sqlContext.sql("select * from Climate_vs_power_consumption_chart"))

summary,avg(KWh),avg(temperature)
Breezy,0.06252927185402184,10.832472151561568
Windy and Overcast,0.1324314024390244,9.881650152439033
Foggy,0.06303930968360498,7.458852105370683
Breezy and Partly Cloudy,0.0754682447684705,11.117179811220394
Partly Cloudy,0.07098884000798618,12.110342307454289
Clear,0.045242388218882254,10.44060325639455
Mostly Cloudy,0.0803520864149503,10.32359365807814
Breezy and Overcast,0.08920285482427849,11.074755820970068
Breezy and Mostly Cloudy,0.07307696878230799,11.166065762735823
Windy,0.07103303417987397,10.48006396792053
